# Création de la base Représentants et Personnes morales 

- Importation des données TC et TITMC 

- Codification des adresses 

- Ajout et traitement du code commune

- Retraiter et simplifier les qualités 

In [2]:
# fonction permettant de traiter les données rep et pm et d'ajouter le code commune insee
import os
import s3fs
import pandas as pd
import functions as fc 

In [ ]:
# Lecture de la base inpi
df_rep_pm = fc.import_inpi("data_rep_pm_2017.csv")

In [ ]:
# nettoyage de diverses variables
df_rep_pm = fc.clean_rep_pm_inpi(df_rep_pm)

In [ ]:
# ajout du code commune
df_rep_pm = fc.add_code_com(df_rep_pm)

In [ ]:
# export de la table
fc.export2minio("df_rep_pm_2017_commune.csv", df_rep_pm)

In [28]:
df_rep_pm = fc.import_inpi( "df_rep_pm_2017_commune.csv", cols=None)

/home/coder/work/data_inpi/code/functions.py:192: DtypeWarning: Columns (14,16,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8",


In [29]:
df_rep_pm.head()

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,...,pays,id_representant,top_civilite,nom_patronymique2,top_particule,qualite2,denomination2,commune,code_departement,code_commune_insee
0,101,15850944,AGIR TECHNOLOGIES,Société par actions simplifiée à associé unique,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,agir technologies,NaN,na,NaN
1,101,15851793,DORAS,Société anonyme à directoire et conseil de sur...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,doras,NaN,na,NaN
2,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Président,VATIN,NaN,Gerard,P.Physique,1947-10-07,...,FRANCE,1.0,False,VATIN,False,président,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0
3,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,NaN,Jacques,P.Physique,1930-04-03,...,FRANCE,3.0,False,VATIN,False,directeur général,societe immobiliere financiere d etudes et de ...,paris,75,NaN
4,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,JORDENS,VATIN,Regina,P.Physique,1948-08-07,...,FRANCE,4.0,False,JORDENS,False,directeur général,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0


In [30]:
# on filtre on ne va garder que les lignes avec le nom patronymique, le nom usage, les prénoms, ou la date de naissance
df_rep_pm = df_rep_pm.loc[(df_rep_pm["nom_patronymique"].isna() == False) | (df_rep_pm["prenoms"].isna() == False) | \
                            (df_rep_pm["date_naissance"].isna() == False), :]

In [6]:
df_rep_pm.shape # 8 671 244

(8671244, 27)

In [31]:
df_rep_pm["siren"].nunique() # 3 977 143

3977143

In [6]:
test = df_rep_pm.loc[df_rep_pm["code_commune_insee"].isna() == False, :]

In [7]:
test.shape # 4 693 722 avec code commune

(4693722, 27)

In [8]:
test = df_rep_pm.loc[df_rep_pm["code_commune"].isna() == False, :]

In [9]:
test.shape  # 7 740 164

(7740164, 27)

In [11]:
df_rep_pm.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite',
       'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance',
       'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3',
       'code_postal', 'ville', 'code_commune', 'pays', 'id_representant',
       'top_civilite', 'nom_patronymique2', 'top_particule', 'qualite2',
       'denomination2', 'commune', 'code_departement', 'code_commune_insee',
       'code_commune3'],
      dtype='object')

In [32]:
# correction pour paris, marseille et lyon
df_rep_pm["code_commune3"] = df_rep_pm["code_commune"]

In [33]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), :].shape # 10249

(10249, 28)

In [34]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"] = "751" + \
    df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), "code_postal"].str.slice(start=4, stop=7)

In [10]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape  # 8 551

(8551,)

In [11]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape # 945

(945,)

In [35]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"] = "6938" + \
    df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_postal"].str.slice(start=5, stop=7)

In [13]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape # 945

(863,)

In [14]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "marseille") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape # 2827

(2827,)

In [36]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "marseille") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"] = "132" + \
    df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "marseille") & (df_rep_pm["code_postal"].isna() == False), "code_postal"].str.slice(start=4, stop=7)

In [12]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "marseille") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape  # 2374

(2374,)

In [37]:
# codifier le type de voie
df_rep_pm["adresse_ligne1_cor"] = df_rep_pm["adresse_ligne1"].str.lower()

In [38]:
df_rep_pm["adresse_ligne1_cor"] = df_rep_pm["adresse_ligne1_cor"].str.strip()

In [39]:
df_rep_pm.loc[df_rep_pm["adresse_ligne1_cor"].isna() == False, "adresse_ligne1_cor"].head()

394          le coin perdu
395          le coin perdu
400     le prince de galle
1050           l'3631 kayl
1286               ch 6312
Name: adresse_ligne1_cor, dtype: object

In [41]:
df_rep_pm["adresse_ligne1_cor"] = (df_rep_pm["adresse_ligne1_cor"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))

In [42]:
df_rep_pm["adresse_ligne1_cor"] = df_rep_pm["adresse_ligne1_cor"].str.replace('"', ' ')

In [43]:
df_rep_pm.loc[df_rep_pm["adresse_ligne1_cor"].isna() == False, "adresse_ligne1_cor"].head()

394          le coin perdu
395          le coin perdu
400     le prince de galle
1050           l'3631 kayl
1286               ch 6312
Name: adresse_ligne1_cor, dtype: object

In [44]:
df_rep_pm["adresse_ligne2_cor"] = df_rep_pm["adresse_ligne2"].str.lower()
df_rep_pm["adresse_ligne2_cor"] = df_rep_pm["adresse_ligne2_cor"].str.strip()
df_rep_pm["adresse_ligne2_cor"] = (df_rep_pm["adresse_ligne2_cor"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))
df_rep_pm["adresse_ligne2_cor"] = df_rep_pm["adresse_ligne2_cor"].str.replace('"', ' ')
df_rep_pm.loc[df_rep_pm["adresse_ligne2_cor"].isna() == False, "adresse_ligne2_cor"].head()

2         1360 chemin saint-roch
3              50 rue de la tour
4         1360 chemin saint-roch
5                 63 rue virgile
13    222 rue jean-marie vianney
Name: adresse_ligne2_cor, dtype: object

In [45]:
df_rep_pm["adresse_ligne3_cor"] = df_rep_pm["adresse_ligne3"].str.lower()
df_rep_pm["adresse_ligne3_cor"] = df_rep_pm["adresse_ligne3_cor"].str.strip()
df_rep_pm["adresse_ligne3_cor"] = (df_rep_pm["adresse_ligne3_cor"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))
df_rep_pm["adresse_ligne3_cor"] = df_rep_pm["adresse_ligne3_cor"].str.replace('"', ' ')
df_rep_pm.loc[df_rep_pm["adresse_ligne3_cor"].isna() == False, "adresse_ligne3_cor"].head()

23    459 route de donsonnas
24    527 route de donsonnas
27                    belley
38          treffort-cuisiat
57              cran gevrier
Name: adresse_ligne3_cor, dtype: object

In [46]:
# Remettre l'adresse dans la bonne colonne
df_rep_pm.loc[(df_rep_pm["adresse_ligne1_cor"].isna()) & 
                (df_rep_pm["adresse_ligne2_cor"].isna() == False), "adresse_ligne1_cor"] = df_rep_pm.loc[(df_rep_pm["adresse_ligne1_cor"].isna()) & 
                (df_rep_pm["adresse_ligne2_cor"].isna() == False), "adresse_ligne2_cor"]

In [47]:
df_rep_pm.loc[(df_rep_pm["adresse_ligne1_cor"].isna()),:].shape # 154 145

(154145, 31)

In [48]:
df_rep_pm.loc[(df_rep_pm["adresse_ligne1_cor"].isna()) & 
                (df_rep_pm["adresse_ligne3_cor"].isna() == False), "adresse_ligne1_cor"] = df_rep_pm.loc[(df_rep_pm["adresse_ligne1_cor"].isna()) & 
                (df_rep_pm["adresse_ligne3_cor"].isna() == False), "adresse_ligne3_cor"]

In [49]:
df_rep_pm.loc[(df_rep_pm["adresse_ligne1_cor"].isna()),:].shape # 69235
# d'autres corrections insee n'ont pas été mises en place pour le moment, voir le code si ces corrections se révèlent nécessaires

(69235, 31)

In [53]:
# Retraiter et simplifier les qualités
import numpy as np
df_rep_pm["qualite_simplifie"] = np.nan

In [54]:
# on reprend la classification de l'Insee
df_rep_pm.loc[df_rep_pm["qualite2"].isin(["actionnaire unique", "associé", "associé actionnaire", "associé commanditaire", 
	"associé en nom", "associé indéfiniment et solidairement responsable", 
	"associé indéfiniment responsable", "associé non exploitant", "associé suppléant", "associé unique", 
	"conjoint marié ou pacsé, associé", "propriétaire du fonds", "propriétaire non exploitant"]), "qualite_simplifie"] = 10

In [55]:
df_rep_pm.loc[df_rep_pm["qualite2"].isin(["associé commandité", "associé commandite", "associé exploitant", "associé gérant", "associé qui participe à la gestion", 
	"associé-gérant", "gérant associé", "gérant associé en nom", "gérant associé indéfiniment responsable", 
	"gérant associé unique", "gérant et associé indéfiniment et solidairement responsable"]), "qualite_simplifie"] = 11

In [59]:
df_rep_pm.loc[df_rep_pm["qualite2"].isin(["administrateur", "administrateur ad'hoc", "administrateur adjoint", 
    "administrateur choisi en raison de ses compétences", "administrateur dont la nomination a été publiée au j.o", 
    "administrateur dont la nomination a été publiée au j.o.", "administrateur délégué", "administrateur délégué adjoint", 
    "administrateur général", "administrateur général adjoint", "administrateur nommé par le csa", 
    "administrateur personnalité qualifiée", "administrateur provisoire", 
    "administrateur représentant des communes et de leurs groupes", "administrateur représentant des locataires", 
    "administrateur représentant des usagers", "administrateur représentant du personnel", "administrateur représentant l'etat", 
    "administrateur représentant la ville de paris", "administrateur représentant le parlement", 
    "administrateur représentant le capital privé", "administrateur représentant les collectivités territoriales", 
    "administrateur représentant les collectivités territoriales", "administrateur représentant les régions", 
    "administrateur représentant les salariés", "administrateur représentant les salariés actionnaires", 
    "administrateur salarié", "administrateur suppléant", "administrateur suppléant représentant l'etat", 
    "administrateur titulaire", "administrateur représentant le capital public", "co-gérant", "cogérant", 
    "directeur", "directeur adjoint", "directeur administratif", "directeur commercial", "directeur d'exploitation", 
    "directeur d'établissement", "directeur des transports", "directeur délégué", "directeur exécutif", "directeur général", 
    "directeur général (non administrateur)", "directeur général adjoint", "directeur général délégué", 
    "directeur général délégué et pharmacien responsable", "directeur général et administrateur", 
    "directeur général non administrateur", "directeur général unique", "directeur gérant", "directeur régional", 
    "directeur technique", "directeur technique et administratif", "dirigeant", "dirigeant de fait", 
    "dirigeant en france d'une personne morale étrangère", "dirigeant responsable", 
    "dirigeant à l'étranger d'une personne morale étrangère", "fondé de pouvoir", 
    "fondé de pouvoir de l'entreprise", "fondé de pouvoir à procuration générale", 
    "fondé de pouvoir à procuration spéciale et individuelle", "gestionnaire", "gouverneur", "gérant", "gérant adjoint", 
    "gérant non associé", "gérant suppléant", "gérant technique", "indivisaire", "mandataire", "mandataire général", "membre du comité collégial", 
    "membre du comité consultatif", "membre du comité d'administration", "membre du comité d'engagement", 
    "membre du comité d'investissement et de gestion", "membre du comité d'orientation", "membre du comité d'orientation stratégique", 
    "membre du comité de contrôle", "membre du comité de coordination", "membre du comité de direction", "membre du comité de gestion", 
    "membre du comité de parrainage", "membre du comité de pilotage", "membre du comité de présidence", "membre du comité de suivi", 
    "membre du comité de supervision", "membre du comité de surveillance", "membre du comité des associés", 
    "membre du comité des investissements", "membre du comité directeur", "membre du comité exécutif", "membre du comité opérationnel", 
    "membre du comité stratégique", "membre du comité stratégique et financier", "membre du conseil", "membre du conseil d'administration", 
    "membre du conseil d'exploitation", "membre du conseil d'orientation", "membre du conseil d'orientation et de surveillance", 
    "membre du conseil de direction", "membre du conseil de gestion", "membre du conseil de gouvernance", "membre du conseil de gérance", 
    "membre du conseil de parrainage", "membre du conseil de présidence", "membre du conseil de surveillance", 
    "membre du conseil de surveillance représentant l'etat", "membre du conseil de surveillance représentant les salariés", 
    "membre du conseil exécutif", "membre du conseil scientifique", "membre du conseil stratégique et de surveillance", 
    "membre du directoire", "membre du g.i.e.", "membre du gie et administrateur", "membre et directeur du directoire", 
    "membre suppléant de la commission de contrôle de gestion", "membre suppléant du ministère de l'économie, de l'industrie", 
    "membre titulaire de la commission de contrôle de gestion", "membre titulaire du ministère de l'économie, de l'industrie", 
    "membre titulaire pers. choisie en raison de ses compétences", "membre titulaire représentant", 
    "membre titulaire représentant des collectivités et personnes", "membre titulaire représentant des personnels de droit privé", 
    "membre titulaire représentant des personnels de droit public", "membre titulaire représentant du conseil d'etat, de la cour", 
    "membre titulaire représentant du ministre chargé de l'enviro", "membre titulaire représentant du ministre chargé des forêts", 
    "personne ayant le pouvoir d'engager à titre habituel l'établ", "personne ayant le pouvoir d'engager à titre habituel la soci", 
    "président", "président directeur général", "président délégué", "président d'honneur", "président directeur général", 
    "président directeur général", "président directeur général et administrateur", 
    "président dont la nomination a été publiée au j.o.", "président du comité", "président du comité d'administration",
    "président du comité d'orientation", "président du comité d'orientation stratégique", "président du comité de contrôle", 
    "président du comité de direction", "président du comité de gestion", "président du comité de suivi", "président du comité de supervision", 
    "président du comité de surveillance", "président du comité des investissements", "président du comité exécutif", 
    "président du comité opérationnel", "président du comité stratégique", "président du comité stratégique et financier",
    "président du conseil", "président du conseil d'administration", "président du conseil d'administration et administrateur", 
    "président du conseil d'administration et directeur général", "président du conseil d'orientation et de surveillance", 
    "président du conseil de direction", "président du conseil de gestion", "président du conseil de gérance", 
    "président du conseil de surveillance", "président du conseil scientifique", "président du conseil stratégique", 
    "président du conseil stratégique et de surveillance", "président du directoire", "président délégué", "président et directeur général", 
    "président gérant", "président suppléant", "société de gestion", "titulaire de la capacité", "trésorier", "trésorier adjoint", 
    "vice gérant", "vice président", "vice président du conseil d'administration", "vice président du conseil de surveillance", 
    "vice-président", "vice-président", "vice-président du comité d'administration", "vice-président du comité de direction", 
    "vice-président du comité de surveillance", "vice-président du comité stratégique", "vice-président du conseil", 
    "vice-président du conseil d'administration", "vice-président du conseil d'orientation et de surveillance", 
    "vice-président du conseil de gérance", "vice-président du conseil de surveillance", 
    "vice-président du conseil stratégique et de surveillance", "vice-président du directoire", 
    "vice-président délégué", "vice-président exécutif", "vice-secrétaire", "vice-trésorier"]), "qualite_simplifie"] = 1

In [60]:
df_rep_pm["qualite_simplifie"].value_counts()
# 1.0     5106566 -> activité de gestion
# 10.0    3142702 -> droit de propriété
# 11.0      14305 -> les deux

1.0     5106566
10.0    3142702
11.0      14305
Name: qualite_simplifie, dtype: int64

In [61]:
df_rep_pm["qualite_simplifie"].isna().sum() # 407 671

407671

In [62]:
df_rep_pm.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite',
       'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance',
       'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3',
       'code_postal', 'ville', 'code_commune', 'pays', 'id_representant',
       'top_civilite', 'nom_patronymique2', 'top_particule', 'qualite2',
       'denomination2', 'commune', 'code_departement', 'code_commune_insee',
       'code_commune3', 'adresse_ligne1_cor', 'adresse_ligne2_cor',
       'adresse_ligne3_cor', 'qualite_simplifie'],
      dtype='object')

In [67]:
# supprimer les doublons avec qualité simplifiée identique
df_rep_pm2 = df_rep_pm.drop_duplicates(subset=["siren", "denomination2", "id_representant", "nom_patronymique2", "prenoms", "qualite_simplifie"])

In [68]:
df_rep_pm2.shape # 8 621 820

(8621820, 32)

In [70]:
df_rep_pm3 = df_rep_pm.drop_duplicates(subset=["siren", "denomination2", "id_representant", "nom_patronymique2", "prenoms"])

In [71]:
df_rep_pm3.shape # 7 384 566

(7384566, 32)

In [73]:
# résumer la qualité des représentants
df_rep_pm2 = df_rep_pm2.loc[:,["siren", "denomination2", "id_representant", "nom_patronymique2", "prenoms", "qualite_simplifie"]].groupby(["siren", "denomination2", "id_representant", "nom_patronymique2", "prenoms"]).agg({"qualite_simplifie" : "sum"})

In [74]:
df_rep_pm2.shape # 7 372 758

(7372758, 1)

In [76]:
df_rep_pm2 = df_rep_pm2.reset_index()

In [77]:
df_rep_pm2.head()

,siren,denomination2,id_representant,nom_patronymique2,prenoms,qualite_simplifie
0,0,capital insurrance,1.0,DERORE,GUY PAUL GEORGES MAURICE,0.0
1,0,capital insurrance,2.0,MUNOZ RAMONET,JULIO,0.0
2,0,creation tradition france le decor de la maiso...,1.0,MICHAELIS,PIERRE,1.0
3,0,creation tradition france le decor de la maiso...,2.0,STELLA,GILBERT,1.0
4,0,creation tradition france le decor de la maiso...,3.0,BEAUDEQUIN,ROSE ANDREE,1.0


In [79]:
df_rep_pm2.loc[df_rep_pm2["siren"] == 0,:].shape # 16

(0, 6)

In [80]:
df_rep_pm2["qualite_simplifie"].value_counts()

1.0     3823156
10.0    1913440
11.0    1240493
0.0      394987
12.0        627
21.0         32
22.0         23
Name: qualite_simplifie, dtype: int64

In [83]:
df_rep_pm2["qualite_simplifie"] = np.minimum(11, df_rep_pm2['qualite_simplifie'])

In [84]:
df_rep_pm2["qualite_simplifie"].value_counts()

1.0     3823156
10.0    1913440
11.0    1241175
0.0      394987
Name: qualite_simplifie, dtype: int64

In [86]:
df_rep_pm4 = pd.merge(df_rep_pm3, df_rep_pm2, on=["siren", "denomination2", "id_representant", "nom_patronymique2", "prenoms"], how="left")

In [87]:
df_rep_pm4.shape # 7384566

(7384566, 33)

In [88]:
df_rep_pm4.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite',
       'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance',
       'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3',
       'code_postal', 'ville', 'code_commune', 'pays', 'id_representant',
       'top_civilite', 'nom_patronymique2', 'top_particule', 'qualite2',
       'denomination2', 'commune', 'code_departement', 'code_commune_insee',
       'code_commune3', 'adresse_ligne1_cor', 'adresse_ligne2_cor',
       'adresse_ligne3_cor', 'qualite_simplifie_x', 'qualite_simplifie_y'],
      dtype='object')

In [89]:
df_rep_pm4 = df_rep_pm4.drop("qualite_simplifie_x", axis=1)

In [90]:
df_rep_pm4 = df_rep_pm4.rename({"qualite_simplifie_y" : "qualite_simplifie"}, axis=1)

In [91]:
df_rep_pm4 = df_rep_pm4.drop(["top_particule", "top_civilite"], axis=1)
df_rep_pm4 = df_rep_pm4.rename({"nom_patronymique2" : "nom_patronymique_cor", "qualite2" : "qualite_cor", "denomination2" : "denomination_cor", 
                                "code_commune3" : "code_commune_cor"}, axis=1)

In [92]:
df_rep_pm4.head()

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,...,qualite_cor,denomination_cor,commune,code_departement,code_commune_insee,code_commune_cor,adresse_ligne1_cor,adresse_ligne2_cor,adresse_ligne3_cor,qualite_simplifie
0,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Président,VATIN,NaN,Gerard,P.Physique,1947-10-07,...,président,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0,84031.0,1360 chemin saint-roch,1360 chemin saint-roch,NaN,1.0
1,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,NaN,Jacques,P.Physique,1930-04-03,...,directeur général,societe immobiliere financiere d etudes et de ...,paris,75,NaN,NaN,50 rue de la tour,50 rue de la tour,NaN,1.0
2,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,JORDENS,VATIN,Regina,P.Physique,1948-08-07,...,directeur général,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0,84031.0,1360 chemin saint-roch,1360 chemin saint-roch,NaN,1.0
3,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,VATIN-CHAIX,Bertrand Heinrich René,P.Physique,1977-06-09,...,directeur général,societe immobiliere financiere d etudes et de ...,strasbourg,67,67482.0,67482.0,63 rue virgile,63 rue virgile,NaN,1.0
4,101,61500740,SOCIETE J.F.B.,Société par actions simplifiée,Président,BENOIT,NaN,Nicolas Arnold Denis,P.Physique,1974-05-15,...,président,societe j.f.b.,ars-sur-formans,14,NaN,1021.0,222 rue jean-marie vianney,222 rue jean-marie vianney,NaN,1.0


In [93]:
fc.export2minio("societe_rep.csv", df_rep_pm4)